<a href="https://colab.research.google.com/github/yota-nagai/combinatorial-optimization-problems/blob/main/12_%E7%84%A1%E5%90%91%E5%B8%B0%E9%82%84%E9%A0%82%E7%82%B9%E9%9B%86%E5%90%88%E5%95%8F%E9%A1%8C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q amplify
from amplify import VariableGenerator, FixstarsClient, solve, Solver
from datetime import timedelta

client = FixstarsClient()
client.token = "******************"  #アクセストークンを入力
client.parameters.timeout = timedelta(milliseconds=2000)  # タイムアウトは 2000 ms


#################### 無向帰還頂点集合問題 ####################

"""
どこがおかしいか、分からない！！！！！
"""

N,M=map(int,input().split())  #頂点数、辺の数
edge=[]  #辺リスト（無向グラフ）
for _ in range(M):
  u,v=map(int,input().split())
  u-=1
  v-=1
  edge.append([u,v])
  edge.append([v,u])

#バイナリ変数の生成
gen=VariableGenerator()
y_v=gen.array("Binary",N)  #頂点vを取り除く（帰還頂点集合に入れる）かどうか
x_vi=gen.array("Binary",shape=(N,N))  #頂点vが高さiであるかどうか
x_uvi=gen.array("Binary",shape=(N,N,N))  #辺(u,v)が高さiであるかどうか

#制約項①　各辺(u,v)について、頂点uと頂点vをともに残した場合に限り辺(u,v)の高さが1つに決定する
f1=0
for u,v in edge:
  cnt=0
  for i in range(N):
    cnt+=x_uvi[u][v][i]+x_uvi[v][u][i]
  f1+=(1-y_v[u])*(1-y_v[v])*(1-cnt)**2

#制約項②　頂点vが残されるなら、vについてただ一つの高さが存在
f2=0
for v in range(N):
  cnt=0
  for i in range(N):
    cnt+=x_vi[v][i]
  f2+=(1-y_v[v]-cnt)**2

#制約項③　高さi>0である頂点vは、高さiである辺（1つ）につながっている
f3=0
for v in range(N):
  for i in range(1,N):
    cnt=0
    for uu,vv in edge:
      if vv==v:
        cnt+=x_uvi[uu][v][i]
    f3+=(x_vi[v][i]-cnt)**2

#制約項④　辺とその両端点の高さが整合しないとダメ
f4=0
for u,v in edge:
  for i in range(1,N):
    #(uの高さがi-1でvの高さがi) or (uの高さがiでvの高さがi-1)のとき1
    flag=x_vi[u][i-1]*x_vi[v][i]+x_vi[u][i]*x_vi[v][i-1]
    f4+=(1-flag)*x_uvi[u][v][i]

#目的関数項　取り除かれる頂点の数を最小化
f5=sum(y_v)

#ペナルティ係数の設定
A,B=map(int,input().split())
f=A*(f1+f2+f3+f4)+B*f5


#ソルバー実行、最適解の取得
result=solve(f,client)
objective=result.best.objective
values=result.best.values

print(objective)